In [30]:
import pandas as pd 
import numpy as np 
import scipy as scp 

from bs4 import BeautifulSoup 
import requests as rq 
import re

from selenium import webdriver

import platform

### inicia_browser()
Uma função que retorna uma variavel do tipo WebDriver
de acordo com o S.O.

### array_df()
Função que pega lista mal estruturada
E retorna uma lista para ser transformada
em DataFrame

### append_manual()
Função que da um append por quantidade pois
É variavil o valor da array que usamos
E evista-se o uso de um for.



In [164]:
def inicia_browser():
    if(platform.system() == 'Windows'):
        browser = webdriver.Chrome(executable_path="../2017-2-projeto-wildcats/chromer/chromedriver.exe")
    else:
        browser = webdriver.Chrome()
    return browser

def array_df(lista):
    array = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range (len(lista)):
        if ':' in lista[i]:
            tops = lista[i].split(':')
            if tops[0] == 'Melhor índice de solução':
                array[5] = tops[1]
            elif tops[0] == 'Melhores Índices de Voltar a Fazer Negócios':
                array[6] = tops[1]
            elif tops[0] == 'Melhores notas médias':
                array[7] = tops[1]
            elif tops[0] == 'Mais resolveram nos últimos 30 dias':
                array[8] = tops[1]
            elif tops[0] == 'Mais resolveram nos últimos 12 meses':
                array[9] = tops[1]
            elif tops[0] == 'Piores empresas nos últimos 30 dias':
                array[10] = tops[1]
            elif tops[0] == 'Mais reclamadas nos últimos 12 meses':
                array[11] = tops[1]
            elif tops[0] == 'Empresas recém-cadastradas com mais reclamações':
                array[12] = tops[1]
            elif tops[0] == 'Mais Reclamadas do dia':
                array[13] = tops[1]
            elif tops[0] == 'Mais Reclamadas da semana':
                array[14] = tops[1]
            elif tops[0] == 'Mais Reclamadas nos últimos 30 dias':
                array[15] = tops[1]
        else:
            if lista[i] != '--':
                array[i] = lista[i]
    return array

            
            
def append_manual(length,lista,array):
    if length == 2:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1]]
    elif length == 3:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2]]
    elif length == 4:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3]]
    elif length == 5:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4]]
    elif length == 6:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5]]
    elif length == 7:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6]]
    elif length == 8:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7]]
    elif length == 9:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7], lista[8]]
    elif length == 10:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7], lista[8], lista[9]]
    elif length == 11:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7], lista[8], lista[9], lista[10]]
    elif length == 12:
        return [array[0] , array[1], array[2], array[3], array[4], lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7], lista[8], lista[9], lista[10], lista[11]]

In [154]:
lista_Links_Tabela = []
url = "https://www.reclameaqui.com.br/ranking/"
browser = inicia_browser()
browser.get(url)
boxes_rankings = browser.find_elements_by_xpath('//div[@class="box-gray"]')
for box in boxes_rankings:
    lista_Empresas_Ranking =box.find_elements_by_xpath('.//li[@class="ng-scope"]')
    #titulo_box = box.find_element_by_xpath("//h2[@class='ng-binding']")
    titulo=box.find_element_by_tag_name('h2')
    numb = 1
    for empresa_no_ranking in lista_Empresas_Ranking:
        empresa = empresa_no_ranking.find_element_by_xpath('.//a[@class="business-name ng-binding ng-scope"]')
        link = empresa.get_attribute("href")
        if(any(link in i for i in lista_Links_Tabela)):
            for i in lista_Links_Tabela:
                if link in i:
                    i.append(titulo.text+':'+str(numb))
                    break;
        else:
            lista_Links_Tabela.append([link,titulo.text+':'+str(numb)])
        numb+=1
browser.close()
browser.quit()

In [162]:
list_p_list_df = []
for t_lks_rnk in lista_Links_Tabela:
    try:
        url_empresa = t_lks_rnk[0]
        browser = inicia_browser()
        browser.get(url_empresa)
        botao = browser.find_elements_by_xpath('//li[@ng-click="getCompanyIndex(\'LAST_THREE_YEARS\')"]')
        botao[1].click()
        box_avaliacoes = browser.find_elements_by_xpath('//div[@class="col-xs-5 col-sm-3 flexbox-col"]')
        nome = browser.find_element_by_xpath('//h1[@class="big-title ng-binding"]').text
        ra = box_avaliacoes[0].text.split("\n")[0]
        vfn = box_avaliacoes[1].text.split("\n")[0]
        isol = box_avaliacoes[2].text.split("\n")[0]
        nota = box_avaliacoes[3].text.split("\n")[0]
        list_p_list_df.append(append_manual(len(t_lks_rnk),t_lks_rnk,[nome,ra,vfn,isol,nota]))
        browser.close()
        browser.quit()
    except ValueError:
        print('Erro durante a raspagem.')
        print(t_lks_rnk)
        print(ValueError)
        print('--------------------------------')
        browser.close()
        browser.quit()

In [167]:
tabela_recaqui = []
for ra_empresas in list_p_list_df:
    tabela_recaqui.append(dict(zip(['nome_empresa','ra','vfn','is','nota','mis','mivfn','mnm','mrutd','mrudm','peutd','mrecudm','ercmr','mrd','mrs','mrecutd'],array_df(ra_empresas))))

In [168]:
df_tabela_recaqui = pd.DataFrame(tabela_recaqui,columns=['nome_empresa','ra','vfn','is','nota','mis','mivfn','mnm','mrutd','mrudm','peutd','mrecudm','ercmr','mrd','mrs','mrecutd'])

In [169]:
df_tabela_recaqui

,nome_empresa,ra,vfn,is,nota,mis,mivfn,mnm,mrutd,mrudm,peutd,mrecudm,ercmr,mrd,mrs,mrecutd
0,Guichê Web,100%,"83,0%","93,5%","8,08",1,2,1,0,0,0,0,0,0,0,0
1,D´Furlan Moveis Planejados,"98,2%","96,5%","97,7%","9,38",2,1,2,0,0,0,0,0,0,0,0
2,FSVAS Participações,"99,9%","65,6%","98,9%","9,06",3,0,7,0,0,0,0,0,0,0,0
3,Hersheys do Brasil,"94,6%","73,0%","78,9%","6,83",4,10,10,0,0,0,0,0,0,0,0
4,Lança Perfume - Loja online,100%,"78,2%","95,6%","7,58",5,0,0,0,0,0,0,0,0,0,0
5,DMCard,"99,5%","90,0%","96,4%","8,87",6,5,3,0,0,0,0,0,0,0,0
6,Ventureshop,"96,4%","92,8%","97,8%","8,58",7,11,0,0,0,0,0,0,0,0,0
7,Quero Bolsa,"99,7%","90,6%","95,7%","9,08",8,9,5,0,0,0,0,0,0,0,0
8,Grupo VR,"99,7%","90,8%","96,2%","8,6",9,7,19,0,0,0,0,0,0,0,0
9,Sidlar,"98,1%","67,3%","95,4%","7,05",10,0,0,0,0,0,0,0,0,0,0


In [171]:
df_tabela_recaqui.to_csv('tabela_recaqui.csv', sep=';', encoding='utf-8',index=False)